In [116]:
import os
import torch
import torchvision

import h5py
import deepdish as dd

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision

import matplotlib.pyplot as plt

from PIL import Image

import time
import pickle as pkl

In [117]:
n_epochs = 25
batch_size_train = 32# was 64
batch_size_test = 32
learning_rate = 0.001
momentum = 0.1
log_interval = 31

random_seed = 1
#torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [118]:
''' 
    NOTES: 
    - it's "No Finding" not "No findings"
    - it's "Pleural_Thickening" not "Pleural_thickening"
    - it's not "Nodule Mass", but rather "Nodule" and "Mass" separately
'''
disease_map = {"Atelectasis" : 0, "Consolidation" : 1, "Infiltration" : 2, "Pneumothorax": 3, "Edema": 4,
               "Emphysema": 5, "Fibrosis": 6, "Effusion" : 7, "Pneumonia" : 8, "Pleural_Thickening" : 9,
               "Cardiomegaly" : 10, "Nodule" : 11, "Mass" : 12, "Hernia" : 13, "No Finding" : 14 }


In [119]:
class GetLoader(torch.utils.data.Dataset):
    def __init__(self, data, view, disease, start, stop, transforms=None):
        
        self.root = os.path.join('data/sorted_images',)
        self.data = data # dict object
        self.transforms = transforms
            
        datalist = self.data[view][disease][start:stop]
        
        self.len_data = len(datalist)
        

        # data objects at the lowest level are stored using this format:
        # {'img_name': '00000011_006.png', 'classes': ['Atelectasis'], 'age': '75', 'gender': 'M'}
        
        for data in datalist:
            
            data['img_path'] = os.path.join(self.root, data['classes'][0], view, data['img_name'])            
            diseases = data['classes']
            one_hot = [0] * 15

            for d in diseases:
                hot_index = disease_map[d]
                one_hot[hot_index] = 1
            
            data['label_tensor'] = torch.Tensor(one_hot)
            
        self.img_paths = []
        self.img_labels = []
        
        for i in range(len(datalist)):
            self.img_paths.append(datalist[i]['img_path'])
            self.img_labels.append(datalist[i]['label_tensor'])

    def __getitem__(self, item):
        
        img_path, label = self.img_paths[item], self.img_labels[item]
  
        try:
            img = (Image.open(img_path))
        except:
            img_path = img_path.replace('/AP/', '/PA/')
            img = (Image.open(img_path))
        #imgs = Image.open(img_paths) # .convert('RGB')
        self.cur_img_path = img_path

        if self.transforms is not None:
            for t in self.transforms:
                img = t(img)

        return img, label
    
    def get_img_path(self):
        return self.cur_img_path

    def __len__(self):
        return self.len_data

### load in datasets

In [120]:
pkl_load = open('dataset.pickle', 'rb')
data = pkl.load(pkl_load)
pkl_load.close()
print("data loaded in successfully")

data loaded in successfully


In [121]:
total = len(data['AP']['Nodule'])
train_start, train_stop = 0, total * 4 // 5
test_start, test_stop = train_stop, -1

print(total)
print(train_start, train_stop, test_start, test_stop)

2154
0 1723 1723 -1


In [122]:
transforms = [torchvision.transforms.Grayscale(), torchvision.transforms.ToTensor()]

In [123]:
dataset_train = GetLoader(data, 'AP', 'Nodule', train_start, train_stop, transforms)
dataset_test = GetLoader(data, 'AP', 'Nodule', test_start, test_stop, transforms)

In [124]:

train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size_train, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size_train, shuffle=True)

print(len(train_loader))
print(len(test_loader))

54
14


In [125]:
class ConvBlock(nn.Module):
    def __init__(self, n_filters_in, n_filters_out, kernel_size=3, stride=1, padding=0): #k=3, s=2, p=0 for down
        super(ConvBlock, self).__init__()
        ops = []
        
        ops.append(nn.Conv2d(n_filters_in, n_filters_out, kernel_size, padding=padding, stride=stride))
        ops.append(nn.BatchNorm2d(n_filters_out))
        ops.append(nn.ReLU(inplace=True))
        
        self.conv = nn.Sequential(*ops)
        
        
    def forward(self, x):
        x = self.conv(x)
        return x

In [144]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = ConvBlock(1, 10, kernel_size=3, stride=2, padding=1) # 1024x1024 -> 512x512
        self.conv2 = ConvBlock(10, 20, kernel_size=3, stride=2, padding=1) # 512x512 -> 256x256
        self.conv3 = ConvBlock(20, 40, kernel_size=3, stride=2, padding=1) # 256x256 -> 128x128
        self.conv4 = ConvBlock(40, 80, kernel_size=3, stride=2, padding=1) # 128x128 -> 64x64
        self.conv5 = ConvBlock(80, 80, kernel_size=3, stride=2, padding=1) # 64x64 -> 32x32
        self.conv_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(80 * 32 * 32, 100)
        self.fc2 = nn.Linear(100, 15)  # changed output to 15 since 15 disease classes
        self.sigmoid = nn.Sigmoid()
    
    
    # hook for the gradients of the activations
    def activations_hook(self, grad):
        self.gradients = grad
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        # Final feature reduction - gradient map goes here
        #if self.train and x.requires_grad:
            #h = x.register_hook(self.activations_hook)
        
        x = self.conv5(x)
        x = self.conv_drop(x)
        
        x = x.view(-1, 20 * 64 * 64)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x
    
    def get_activations(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        return x

In [145]:
network = Net()
#network = torchvision.models.vgg11(pretrained=False)
network.fc = nn.Linear(512, 15)
network = torch.nn.DataParallel(network)
network.cuda()
optimizer = optim.SGD(network.parameters(), lr=learning_rate,
                      momentum=momentum)
print('Trainable params: {}'.format(sum(p.numel() for p in network.parameters() if p.requires_grad)))

Trainable params: 8297490


In [146]:
def train(epoch):

    network.train()
    train_losses = []
    correct = 0
    num_tested = 0
    net_loss = 0
    
    for batch_idx, (data, target) in enumerate(train_loader):  # (output - actual )
        
        if torch.cuda.is_available:
            data, target = data.cuda(), target.cuda()

        output = network(data)
        
        criterion = torch.nn.BCELoss()  # 1 0 1 0 0 0 
        loss = criterion(output, target)
        loss.backward()
        
        net_loss += loss.detach().cpu().item()
        train_losses.append(loss.item())

        num_tested += len(data)
        
        if batch_idx == len(train_loader) - 1:
            optimizer.zero_grad()
            optimizer.step()
        
        #if(batch_idx == 1):
            #print(output)
            
        for p in output:
            p = p.data

            for i in range(len(p)):
                if p[i] > 0.5:
                    p[i] = 1
                else:
                    p[i] = 0
            torch.tensor(p)
            
        correct = torch.sum(torch.tensor(output == target)).sum()
        
    net_loss /= len(train_loader)
    return net_loss, correct


In [147]:
def test():
    
    network.eval()
    test_losses = []
    net_loss = 0
    correct = 0
    
    num_tested = 0
    
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            if torch.cuda.is_available:
                data, target = data.cuda(), target.cuda()
                
            if batch_idx == 50:
                break;
            else:
                num_tested += len(data)
            
            output = network(data)
            criterion = torch.nn.BCELoss()
            net_loss += criterion(output, target).item()

            pred = output.clone().cpu()
            target = target.cpu()
            
            for p in pred:
                p = p.data

                for i in range(len(p)):
                    if p[i] > 0.5:
                        p[i] = 1
                    else:
                        p[i] = 0
                torch.tensor(p)


            correct = torch.sum(torch.tensor(pred == target)).sum()
        
    net_loss /= len(test_loader)
    return net_loss, correct


In [148]:
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))
print(torch.cuda.is_available())

1
GeForce RTX 2080 Ti
True


In [149]:
train_sample_size = len(train_loader) * batch_size_train
test_sample_size = len(test_loader) * batch_size_test
print(train_sample_size, test_sample_size)

1728 448


In [150]:
print("Starting Training...")
test()
for epoch in range(1, n_epochs + 1):
    start_time = time.time()

    loss_train, n_correct_train = train(epoch)
    loss_test, n_correct_test = test()
    
    elapsed_time = time.time() - start_time
    print("-----------Epoch " + str(epoch) + " (time = ", round(elapsed_time, 2), "s) ----------------")
    print('\nTest set:  Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        loss_test, n_correct_test, test_sample_size, 100. * n_correct_test / test_sample_size))
    print('Train set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        loss_train, n_correct_train, train_sample_size, 100. * n_correct_train / train_sample_size))
    

Starting Training...


/usr/remote/anaconda-3.7-2020-05-28/lib/python3.7/site-packages/ipykernel_launcher.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/remote/anaconda-3.7-2020-05-28/lib/python3.7/site-packages/ipykernel_launcher.py:38: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/remote/anaconda-3.7-2020-05-28/lib/python3.7/site-packages/ipykernel_launcher.py:40: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/remote/anaconda-3.7-2020-05-28/lib/python3.7/site-packages/ipykernel_launcher.py:42: UserWarning: To copy construct from a tensor, it is 

-----------Epoch 1 (time =  42.59 s) ----------------

Test set:  Avg. loss: 0.6998, Accuracy: 91/448 (20%)
Train set: Avg. loss: 0.7135, Accuracy: 193/1728 (11%)

-----------Epoch 2 (time =  42.73 s) ----------------

Test set:  Avg. loss: 0.6984, Accuracy: 112/448 (25%)
Train set: Avg. loss: 0.7127, Accuracy: 194/1728 (11%)

-----------Epoch 3 (time =  42.87 s) ----------------

Test set:  Avg. loss: 0.6993, Accuracy: 103/448 (23%)
Train set: Avg. loss: 0.7146, Accuracy: 186/1728 (11%)

-----------Epoch 4 (time =  42.92 s) ----------------

Test set:  Avg. loss: 0.6996, Accuracy: 94/448 (21%)
Train set: Avg. loss: 0.7134, Accuracy: 177/1728 (10%)

-----------Epoch 5 (time =  42.87 s) ----------------

Test set:  Avg. loss: 0.6990, Accuracy: 109/448 (24%)
Train set: Avg. loss: 0.7135, Accuracy: 201/1728 (12%)

-----------Epoch 6 (time =  42.89 s) ----------------

Test set:  Avg. loss: 0.6988, Accuracy: 121/448 (27%)
Train set: Avg. loss: 0.7133, Accuracy: 194/1728 (11%)

-----------E

### verifying components work below

In [156]:
train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size_train, shuffle=True)
img, label = next(iter(train_loader))
print("displaying label vector vs network output:")
print(label[0])
pred = network(img)
print(pred[0].cpu().data)


displaying label vector vs network output
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.])
tensor([0.4621, 0.4836, 0.5350, 0.5196, 0.4424, 0.5067, 0.4118, 0.5184, 0.4518,
        0.4963, 0.5426, 0.4930, 0.4911, 0.5354, 0.4615])


In [159]:
print("displaying accumulation of loss over part of a batch")
for p in pred:

    for i in range(len(p)):
        if p[i] > 0.5:
            p[i] = 1
        else:
            p[i] = 0
#print(pred[0].cpu().data)

criterion = nn.BCELoss()
loss = 0
for i in range(5):
    loss += criterion(pred[i].cuda(), label[i].cuda())
    print("sum of loss:", loss)
print("avg loss: ", loss / 5)

displaying accumulation of loss over part of a batch
sum of loss: tensor(12.8945, device='cuda:0', grad_fn=<AddBackward0>)
sum of loss: tensor(27.6310, device='cuda:0', grad_fn=<AddBackward0>)
sum of loss: tensor(42.3676, device='cuda:0', grad_fn=<AddBackward0>)
sum of loss: tensor(55.2620, device='cuda:0', grad_fn=<AddBackward0>)
sum of loss: tensor(73.6827, device='cuda:0', grad_fn=<AddBackward0>)
avg loss:  tensor(14.7365, device='cuda:0', grad_fn=<DivBackward0>)


In [160]:
print(len(train_loader))
print(len(pred))
print(len(pred[0]))

54
32
15
